In [57]:
import pandas as pd 
import numpy as np 
import os

from utils import * 

df_2012_summer = pd.DataFrame(pd.read_csv(os.path.join("/kaggle", Y_2012, "summer.csv")))
df_2012_winter = pd.DataFrame(pd.read_csv(os.path.join("/kaggle", Y_2012, "winter.csv")))
df_2012 = pd.concat([df_2012_summer, df_2012_winter])
df_2012 = df_2012.drop('Athlete', axis=1)
df_2016 = pd.DataFrame(pd.read_csv(os.path.join("/kaggle", Y_2016, "athletes.csv")))
df_2020 = pd.DataFrame(pd.read_csv(os.path.join("/kaggle", Y_2020, "medals.csv")))

# print(f'df_2012:\n{df_2012.head()} \n df_2016:\n{df_2016.head()} \n df_2020:\n{df_2020.head()}')

In [58]:
df_2012.head()

,Year,City,Sport,Discipline,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,GRE,Men,100M Freestyle For Sailors,Silver


# year, country, gold_medal, silver_medal, bronze_medal, total

In [60]:
df_2020['gold_medal']=df_2020['medal_code'].apply(lambda x: 1 if x==1 else 0)
df_2020['silver_medal']=df_2020['medal_code'].apply(lambda x: 1 if x==2 else 0)
df_2020['bronze_medal']=df_2020['medal_code'].apply(lambda x: 1 if x==3 else 0)
# df_2020['year'] = 2020
df_2020 = df_2020[['country_code', 'gold_medal', 'silver_medal', 'bronze_medal']]
df_2020 = df_2020.groupby('country_code').agg(sum)
df_2020 = df_2020.reset_index(drop=False)
df_2020.insert(0, 'year', 2020)
df_2020['total'] = df_2020['gold_medal'] + df_2020['silver_medal'] + df_2020['bronze_medal']
df_2020.head()

,year,country_code,gold_medal,silver_medal,bronze_medal,total
0,2020,ARG,0,18,25,43
1,2020,ARM,0,2,2,4
2,2020,AUS,36,27,66,129
3,2020,AUT,1,1,5,7
4,2020,AZE,0,3,4,7


In [61]:
df_2016 = df_2016.rename({'nationality': 'country_code', 'gold': 'gold_medal', 'silver': 'silver_medal', 'bronze': 'bronze_medal'}, axis=1)
df_2016 = df_2016[['country_code', 'gold_medal', 'silver_medal', 'bronze_medal']]
df_2016 = df_2016.groupby('country_code').agg(sum)
df_2016.reset_index(drop=False)
df_2016.insert(0, 'year', 2016)
df_2016['total'] = df_2016['gold_medal'] + df_2016['silver_medal'] + df_2016['bronze_medal']
df_2016.head()

,year,gold_medal,silver_medal,bronze_medal,total
country_code,,,,,
AFG,2016,0,0,0,0
ALB,2016,0,0,0,0
ALG,2016,0,2,0,2
AND,2016,0,0,0,0
ANG,2016,0,0,0,0


In [64]:
df_2012.Year.unique()

array([1896, 1900, 1904, 1908, 1912, 1920, 1924, 1928, 1932, 1936, 1948,
       1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992,
       1996, 2000, 2004, 2008, 2012, 1994, 1998, 2002, 2006, 2010, 2014])